In [2]:
aolQueries = []
knowHowQueries = []
wikiHowQueries = []

In [3]:
for i in range(1, 11):
  if i < 10:
    fileName = "drive/MyDrive/EDDS/user-ct-test-collection-0" + str(i) + ".txt"
  else:
    fileName = "drive/MyDrive/EDDS/user-ct-test-collection-" + str(i) + ".txt"

  lines = []
  with open(fileName) as f:
    lines = f.readlines()

  count = 0
  for line in lines:
    if count > 0:
      query = line.split("\t")[1]
      aolQueries.append(query)

    count += 1

In [4]:
print(aolQueries[:100])

['rentdirect.com', 'www.prescriptionfortime.com', 'staple.com', 'staple.com', 'www.newyorklawyersite.com', 'www.newyorklawyersite.com', 'westchester.gov', 'space.comhttp', 'dfdf', 'dfdf', 'vaniqa.comh', 'www.collegeucla.edu', 'www.elaorg', '207 ad2d 530', '207 ad2d 530', 'broadway.vera.org', 'broadway.vera.org', 'vera.org', 'broadway.vera.org', 'frankmellace.com', 'ucs.ljx.com', 'attornyleslie.com', 'merit release appearance', 'www.bonsai.wbff.org', 'loislaw.com', 'rapny.com', 'whitepages.com', 'lottery', 'lottery', 'ameriprise.com', 'susheme', 'united.com', 'mizuno.com', "p; .; p;' p; ' ;' ;';", "p; .; p;' p; ' ;' ;';", 'asiansexygoddess.com', 'buddylis', 'bestasiancompany.com', 'lottery', 'lottery', 'ask.com', 'weather.com', 'wellsfargo.com', 'www.tabiecummings.com', 'wanttickets.com', 'yahoo.com', '-', 'www.ngo-quen.org', '-', 'vietnam', 'vietnam', 'vietnam', 'vietnam', 'vietnam', 'vietnam', '-', 'myspace.co', 'myspace.com', 'googl', 'chasebadkids.net', 'ozark horse blankets', 'www.

In [5]:
aolTopSuffixes = {}
aolTop2WordSuffixes = {}
aolTop3WordSuffixes = {}

for query in aolQueries:
  words = query.split(" ")
  lastOne = words[-1]

  if lastOne in aolTopSuffixes:
    aolTopSuffixes[lastOne] += 1
  else:
    aolTopSuffixes[lastOne] = 1


  if len(words) >= 2:
    lastTwo = words[-2] + " " + words[-1]

    if lastTwo in aolTop2WordSuffixes:
      aolTop2WordSuffixes[lastTwo] += 1
    else:
      aolTop2WordSuffixes[lastTwo] = 1

  if len(words) >= 3:
    lastThree = words[-3] + " " + words[-2] + " " + words[-1]
    
    if lastThree in aolTop3WordSuffixes:
      aolTop3WordSuffixes[lastThree] += 1
    else:
      aolTop3WordSuffixes[lastThree] = 1

In [11]:
print(list(aolTop3WordSuffixes.items())[:50])

[('207 ad2d 530', 2), ('merit release appearance', 1), ("' ;' ;';", 2), ('ozark horse blankets', 1), ('in pool-32148876 n00', 4), ('pop up adds', 3), ('childs wonderland company', 2), ("child's wonderland company", 2), ('grand rapids michigan', 364), ('childs wonderland co.', 1), ("child's wonderland co.", 1), ('budget truck rental', 87), ('holiday mansion houseboat', 2), ('to the future', 188), ('jet blue airlines', 818), ('coats tire equipment', 4), ('select business services', 1), ('business services title', 1), ('estate settlement services', 6), ('and settlement services', 6), ('american lenders advantage', 1), ('first american chesapeake', 1), ('chesapeake title services', 1), ('united one resources', 1), ('plus solutions group', 3), ('search and abstract', 1), ('integrated real estate', 2), ('integrated loan services', 1), ('date of birth', 245), ('auto locator pennsylvania', 2), ('kentucky fried chicken', 168), ('fried chicken menu', 14), ('black orange rectangle', 1), ('wnmu ho

In [43]:
aolTopSuffixesCombined = {**aolTop2WordSuffixes, **aolTop3WordSuffixes}

aolTopSuffixesSorted = [*dict(sorted(aolTopSuffixesCombined.items(), key=lambda item: item[1]))]
aolTopSuffixesSorted.reverse()

#aolTop2WordSuffixesSorted = [*dict(sorted(aolTop2WordSuffixes.items(), key=lambda item: item[1]))]
#aolTop2WordSuffixesSorted.reverse()

#aolTop3WordSuffixesSorted = [*dict(sorted(aolTop3WordSuffixes.items(), key=lambda item: item[1]))]
#aolTop3WordSuffixesSorted.reverse()

In [40]:
print(aolTopSuffixesSorted[:50])

['-', 'google', 'com', 'ebay', 'yahoo', 'lyrics', 'pictures', '.com', 'yahoo.com', 'games', 'myspace', 'mapquest', 'sale', 'google.com', 'florida', 'myspace.com', 'for sale', 'pics', 'school', 'sex', 'news', 'airlines', 'texas', 'parts', 'photos', 'college', 'porn', 'bank', 'lottery', 'jobs', 'center', 'cards', 'pages', 'girls', 'recipes', 'furniture', 'ny', 'ohio', '2006', 'internet', 'music', 'www.yahoo.com', 'codes', 'america', 'york', 'www.google.com', 'weather', 'art', 'new york', 'california']


In [47]:
def findTopNCompletions(topSuffixesSorted, query, n):
  completions = []
  nCompletions = 0

  for suffix in topSuffixesSorted:
    if suffix.startswith(query.split("_")[-1]):
      completions.append(query.rsplit("_", 1)[0] + "_" + suffix.replace(" ", "_"))
      nCompletions += 1

    if nCompletions >= n:
      break

  return completions

In [49]:
print(findTopNCompletions(aolTopSuffixesSorted, "getting_organized_at_work", 20))

['getting_organized_at_work_from_home', 'getting_organized_at_work_at_home', 'getting_organized_at_workers_compensation', 'getting_organized_at_work_boots', 'getting_organized_at_work_place', 'getting_organized_at_workforce_commission', 'getting_organized_at_workforce_development', 'getting_organized_at_works_cited', 'getting_organized_at_work_force', 'getting_organized_at_work_sheets', 'getting_organized_at_working_from_home', 'getting_organized_at_work_day', 'getting_organized_at_workmans_comp', 'getting_organized_at_work_shoes', 'getting_organized_at_work_source', 'getting_organized_at_work_out', 'getting_organized_at_work_in_texas', 'getting_organized_at_workers_comp', 'getting_organized_at_work_environment', 'getting_organized_at_work_for']


In [50]:
print(findTopNCompletions(aolTopSuffixesSorted, "choose_bathroom", 20))

['choose_bathroom_vanities', 'choose_bathroom_sinks', 'choose_bathroom_accessories', 'choose_bathroom_vanity', 'choose_bathroom_cabinets', 'choose_bathroom_design', 'choose_bathroom_fixtures', 'choose_bathroom_tile', 'choose_bathroom_rugs', 'choose_bathroom_designs', 'choose_bathroom_furniture', 'choose_bathroom_remodeling', 'choose_bathroom_faucets', 'choose_bathroom_remodel_pictures', 'choose_bathroom_mirrors', 'choose_bathroom_tiles', 'choose_bathroom_lighting', 'choose_bathroom_makeover', 'choose_bathroom_decorating', 'choose_bathroom_ideas']


In [51]:
print(findTopNCompletions(aolTopSuffixesSorted, "low_wedding_budget", 20))

['low_wedding_budget_car_rental', 'low_wedding_budget_rental_car', 'low_wedding_budget_rental', 'low_wedding_budget_rental_cars', 'low_wedding_budget_truck_rental', 'low_wedding_budget_worksheet', 'low_wedding_budget_car_rentals', 'low_wedding_budget_forms', 'low_wedding_budget_inn', 'low_wedding_budget_blinds', 'low_wedding_budget_cuts', 'low_wedding_budget_sheet', 'low_wedding_budget_car_sales', 'low_wedding_budget_travel', 'low_wedding_budget_planning', 'low_wedding_budget_decorating', 'low_wedding_budget_plan', 'low_wedding_budget_rentals', 'low_wedding_budget_suites', 'low_wedding_budget_rent_car']
